In [1]:
import pandas as pd
import os
import numpy as np
from utils.data_process import sample_data

In [2]:
sample_size = 150

In [4]:
x = []
y = []
# if True:
for file in os.listdir("data/preds"):
    df = pd.read_csv(f"data/preds/{file}")
    pulse = np.array(df["pulse"])
    pulse, num_samples = sample_data(pulse)
    print(np.shape(pulse))
    x.extend(pulse)
    y.extend([[df["sys"][0], df["dia"][0]]]*num_samples)
  
x = np.array(x)
y = np.array(y)
print(np.shape(x))
print(np.shape(y))


(3, 150)
(6, 150)
(5, 150)
(12, 150)
(12, 150)
(6, 150)
(9, 150)
(6, 150)
(59, 150)
(59, 2)


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)


In [4]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(150, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(2))  # 2 outputs for systolic and diastolic pressure

model.compile(optimizer='adam', loss='mse')


In [5]:
epochs = 20
batch_size = 10
history = model.fit(x, y, epochs=epochs, batch_size=batch_size)

Epoch 1/20

6/6 [==============================] - 6s 35ms/step - loss: 9964.5146
Epoch 2/20
6/6 [==============================] - 0s 29ms/step - loss: 9448.0176
Epoch 3/20
6/6 [==============================] - 0s 29ms/step - loss: 7190.9512
Epoch 4/20
6/6 [==============================] - 0s 30ms/step - loss: 3363.5369
Epoch 5/20
6/6 [==============================] - 0s 30ms/step - loss: 734.2788
Epoch 6/20
6/6 [==============================] - 0s 27ms/step - loss: 241.5815
Epoch 7/20
6/6 [==============================] - 0s 32ms/step - loss: 429.7001
Epoch 8/20
6/6 [==============================] - 0s 32ms/step - loss: 177.0738
Epoch 9/20
6/6 [==============================] - 0s 29ms/step - loss: 119.6707
Epoch 10/20
6/6 [==============================] - 0s 29ms/step - loss: 125.3106
Epoch 11/20
6/6 [==============================] - 0s 36ms/step - loss: 104.6075
Epoch 12/20
6/6 [==============================] - 0s 33ms/step - loss: 77.1002
Epoch 13/20
6/6 [================

In [9]:
y_pred = model.predict(x_test)

1/1 [==============================] - 1s 952ms/step


In [12]:
print(np.shape(x_test))
print(type(x_test))

(12, 150)
<class 'numpy.ndarray'>


In [10]:
print(y_pred)

[[116.27044   78.04585 ]
 [116.79061   78.41692 ]
 [119.126045  80.06535 ]
 [118.5578    79.70346 ]
 [118.24      79.5693  ]
 [116.097664  77.991104]
 [118.23768   79.492134]
 [118.17768   79.50893 ]
 [116.60473   78.26906 ]
 [120.05093   80.76922 ]
 [119.327995  80.351074]
 [116.76719   78.39822 ]]


In [6]:
model.save("model.keras")